In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
import json


kaggle_json = './drive/MyDrive/kaggle.json'
with open(kaggle_json, 'r') as f:
    json_data = json.load(f)
    os.environ['KAGGLE_USERNAME'] = json_data['username']
    os.environ['KAGGLE_KEY'] = json_data['key']


In [3]:
# 데이터 다운로드(api key 세팅 참조 링크 : https://github.com/Kaggle/kaggle-api)
!kaggle competitions download -c aerial-cactus-identification

  0% 0.00/12.0M [00:00<?, ?B/s]
100% 12.0M/12.0M [00:00<00:00, 132MB/s]


In [4]:
import os

# 폴더 생성 (기존재시 pass)
os.makedirs('./input/train',  exist_ok=True)
os.makedirs('./input/test',  exist_ok=True)
os.makedirs('./output', exist_ok=True)

In [5]:
# 압축 해제 후 input 에 train.csv test.csv  넣기
import zipfile
zipfile.ZipFile(f'aerial-cactus-identification.zip').extractall('./input')

In [6]:
# 추가로 압축해제
zipfile.ZipFile(f'./input/test.zip').extractall('./input')
zipfile.ZipFile(f'./input/train.zip').extractall('./input')

In [7]:
import torch # 파이토치
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed) # 파이썬 난수 생성기 시드 고정
np.random.seed(seed) # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed) # 파이토치 난수 생성기 시드 고정(CPU 사용시)
torch.cuda.manual_seed(seed) # 파이토치 난수 생성기 시드 고정(GPU 사용시)
torch.cuda.manual_seed_all(seed) # 파이토치 난수 생성기 시드 고정(멀티 GPU 사용시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False # cudnn 사용해제

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
import pandas as pd

# 데이터 경로
labels = pd.read_csv('./input/train.csv')
submission = pd.read_csv('./input/sample_submission.csv')

In [10]:
from sklearn.model_selection import train_test_split

# 훈련데이터, 검증 데이터 분리
train, valid = train_test_split(labels,
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=50)

In [11]:
print('훈련 데이터 개수 :', len(train))
print('검증 데이터 개수 :', len(valid))

훈련 데이터 개수 : 15750
검증 데이터 개수 : 1750


In [12]:
import cv2 # openCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
  # 초기화 메서드(생성자)
  def __init__(self, df, img_dir='./', transform=None):
    super().__init__() # 상속받은 Dataset 생성자 호출
    # 전달받은 인수들 저장 
    self.df = df
    self.img_dir = img_dir
    self.transform = transform

  # 데이터셋 크기 반환 메서드
  def __len__(self):
    return len(self.df)

  # 인덱스(idx)에 해당하는 데이터 반환 메서드
  def __getitem__(self, idx):
    img_id = self.df.iloc[idx, 0] # 이미지 ID
    img_path = self.img_dir + img_id # 이미지 파일 경로
    image = cv2.imread(img_path) # 이미지 파일 읽기
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # 이미지 색상 보정
    label = self.df.iloc[idx, 1] # 이미지 레이블 (타깃값)

    if self.transform is not None:
      image = self.transform(image) # 변환기가 있다면 이미지 변환

    return image, label 


In [13]:
from torchvision import transforms # 이미지 변환을 위한 모듈

transform = transforms.ToTensor()

In [14]:
dataset_train = ImageDataset(df = train, img_dir = './input/train/', transform = transform)
dataset_valid = ImageDataset(df = valid, img_dir = './input/train/', transform = transform)

In [15]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

**모델 생성**

In [16]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

In [17]:
class Model(nn.Module):
  # 신경망 계층 정의
  def __init__(self):
    super().__init__() # 상속받은 nn.Module 의 __init__() 메서도 호출

    # 첫 번째 합성곱 계층
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)

    # 두 번째 합성곱 계층
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)

    # 최대 풀링 계층
    self.max_pool = nn.MaxPool2d(kernel_size=2)

    # 평균 풀링 계층
    self.avg_pool = nn.AvgPool2d(kernel_size=2)

    # 전결합 계층
    self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)

  # 순전파 출력 정의
  def forward(self, x):
    x = self.max_pool(F.relu(self.conv1(x)))
    x = self.avg_pool(F.relu(self.conv2(x)))
    x = self.avg_pool(x)
    x = x.view(-1, 64 * 4 * 4) # 평탄화
    x = self.fc(x)
    return x

In [18]:
model = Model().to(device)
model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

In [19]:
# 손실 함수 설정
criterion = nn.CrossEntropyLoss()

In [20]:
# 옵티마이저 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [21]:
epochs = 10 # 총 에폭
# 총 에폭만큼 반폭 

for epoch in range(epochs):
  epoch_loss = 0 # 에폭별 손실값 최소화

  # '반복 횟수' 만큼 반복
  for images, labels in loader_train:
    # 이미지, 레이블 데이터 미니배치를 장비에 할당
    images = images.to(device)
    labels = labels.to(device)

    # 옵티마이저 내 기울기 초기화
    optimizer.zero_grad()

    # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
    outputs = model(images)

    # 손실 함수를 활용해 outpus 와 labels 의 손실값 계산
    loss = criterion(outputs, labels)

    # 현재 배치에서의 손실값 추가
    epoch_loss += loss.item() # 역전파 수행
    loss.backward()
    # 가중치 갱신
    optimizer.step()

  # 훈련 데이터 손실값 출력
  print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss / len(loader_train):.4f}')

에폭 [1/10] - 손실값: 0.5403
에폭 [2/10] - 손실값: 0.3786
에폭 [3/10] - 손실값: 0.2664
에폭 [4/10] - 손실값: 0.2207
에폭 [5/10] - 손실값: 0.1957
에폭 [6/10] - 손실값: 0.1861
에폭 [7/10] - 손실값: 0.1747
에폭 [8/10] - 손실값: 0.1641
에폭 [9/10] - 손실값: 0.1543
에폭 [10/10] - 손실값: 0.1494


In [22]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수 임포트 

# 실젯값 과 예측 확률값을 담을 리스트 
true_list = []
preds_list = []

In [23]:
model.eval()

with torch.no_grad():
  for images, labels in loader_valid:
    # 이미지, 레이블 데이터 미니배치를 장비에 할당
    images = images.to(device)
    labels = labels.to(device)

    # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 줄력값 계산
    outputs = model(images)
    preds = torch.softmax(outputs.cpu(), dim=1)[:,1]
    true = labels.cpu()

    # 예측확률과 실젯값을 리스트에 추가
    preds_list.extend(preds)
    true_list.extend(true)

print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')


검증 데이터 ROC AUC : 0.9874


In [24]:
dataset_test = ImageDataset(df=submission, img_dir ='./input/test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

In [25]:
model.eval()

preds = [] # 타깃 예측값 저장용 리스트 초기화

with torch.no_grad():
  for images, _ in loader_test:
    # 이미지 데이터 미니배치를 장비에 할당
    images = images.to(device)

    # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 줄력값 계산
    outputs = model(images)
    # 타깃값 1일 확률 
    preds_part = torch.softmax(outputs.cpu(), dim=1)[:,1].tolist()

    # 예측확률 리스트에 추가
    preds.extend(preds_part)


In [26]:
submission['has_cactus'] = preds
submission.to_csv('./output/submission.csv', index=False)

In [29]:
!kaggle competitions submit -c aerial-cactus-identification -f ./output/submission.csv -m submit_in_colab

100% 219k/219k [00:01<00:00, 127kB/s]
400 - Bad Request
